# Use differences as additional output but scaled

This file is for using the difference as addaitional output. We use the input voltage to predict the electrical power as well as the difference in power. We can predict the curretn difference but we need to take multiple inputs into account for a RNN to work properly.

The difference to the approach in file (23_) is we scale the difference again to [-1, 1] before applying the model.

## Preliminaries

### Import Libraries

In [1]:
# import own libraries
import Global_Functions as gf
import Neuronal_Networks as nn

# import python libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time
timestr = time.strftime("%Y-%m-%d_%H-%M/")

# import tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.callbacks import EarlyStopping

In [2]:
np.random.seed(123)

### Open files

In [3]:
OPEN_FOLDER = '../Data/Preped_Data/'

In [4]:
ex_1 = gf.open_CSV_file('experiment_1_short.csv', OPEN_FOLDER)
ex_4 = gf.open_CSV_file('experiment_4_short.csv', OPEN_FOLDER)
ex_9 = gf.open_CSV_file('experiment_9_short.csv', OPEN_FOLDER)
ex_20 = gf.open_CSV_file('experiment_20_short.csv', OPEN_FOLDER)
ex_21 = gf.open_CSV_file('experiment_21_short.csv', OPEN_FOLDER)
ex_22 = gf.open_CSV_file('experiment_22_short.csv', OPEN_FOLDER)
ex_23 = gf.open_CSV_file('experiment_23_short.csv', OPEN_FOLDER)
ex_24 = gf.open_CSV_file('experiment_24_short.csv', OPEN_FOLDER)

In [5]:
experiments = [ex_1, ex_4, ex_9, ex_20, ex_21, ex_22, ex_23, ex_24]
names = ['1', '4', '9', '20', '21','22', '23', '24']

In [6]:
OPEN_SYNTHETIC = '../Data/Synthetic_Data/Without_Noise/Sample_Ratio_1/quadratic_40_0_linear_1'

In [7]:
experiments = gf.load_synthetic(OPEN_SYNTHETIC, length = 200)
names = [str(x+1) for x in range(len(experiments)-1)]
names.append("hand")

### Specify parameters

In [8]:
NEUR = 512
EPOCH = 200
LAG = 60
DIFFERENCE = "predict_scaled"
BATCH_SIZE = 512

In [9]:
train = "150"
val = "30"

train_exs = experiments[:150]
ex_train = gf.use_multiple_experiments(train_exs)
ex_val = gf.use_multiple_experiments(experiments[150:180])
ex_test = gf.use_multiple_experiments(experiments[180:200])

In [10]:
image_path = "../Images/Validation/Synthetic/Difference/" + str(DIFFERENCE) + "/" + timestr
image_folder = image_path
gf.check_folder(image_folder)

Creation of directory ../Images/Validation/Synthetic/Difference/predict_scaled/2021-05-28_11-31/ successful.


In [11]:
# saving folder for images
model_path = "../Models/Validation/Synthetic/Difference/" + str(DIFFERENCE) + "/" + timestr
model_folder = model_path

gf.check_folder(model_folder)

Creation of directory ../Models/Validation/Synthetic/Difference/predict_scaled/2021-05-28_11-31/ successful.


In [12]:
specs = {
    'neurons': NEUR,
    'epochs': EPOCH,
    'lag': LAG,
    'difference': DIFFERENCE,
    'batch_size': BATCH_SIZE,
    'model': "LSTM(Neur), Dense(Neur/2), Dense(Out)",
    'data': 'Synthetic - (40, 0, 1)',
    'data_points': '150 for training - 30 for validation',
    'sampling_rate': 1,
    'loss_function': 'Adagrad'
}

specs_str = ""
for cat, descr in specs.items():
    specs_str += f'{cat}: {descr} \n'

with open(image_folder + "00_specs.txt", "w") as text_file:
    text_file.write(specs_str)

## Training the model

In [13]:
model, history, scaler_train, X_train, y_train, _, X_val, y_val = nn.train_model(ex_train, ex_val, save_folder = model_folder,
                                                                                 difference_chosen = DIFFERENCE, lag_chosen = LAG,
                                                                                 nmb_epochs = EPOCH, neurons_chosen = NEUR, batch_size = BATCH_SIZE)

Folder already exists.


C:\Users\FlorianLeiser\Documents\Masterarbeit\Code\Data_Processing.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['diff'] = diffs_scaled
C:\Users\FlorianLeiser\Documents\Masterarbeit\Code\Data_Processing.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['diff'] = diffs_scaled


Epoch 10 of 200 is done.
Epoch 20 of 200 is done.
Epoch 30 of 200 is done.
Epoch 40 of 200 is done.
Epoch 50 of 200 is done.
Epoch 60 of 200 is done.
Epoch 70 of 200 is done.
Epoch 80 of 200 is done.
Epoch 90 of 200 is done.



KeyboardInterrupt



In [ ]:
all_scaler = list()
all_X_values = list()
all_y_values = list()
all_predictions = list()
all_scaled_predictions = list()
results = pd.DataFrame(index = names, columns = ['rmse', 'r2', 'mae', 'maxae', 'rmse_scaled', 'r2_scaled', 'mae_scaled', 'maxae_scaled'])

for i in range(len(experiments)):
    rms, r2, mae, maxae = [], [], [], []
    rms_scaled, r2_scaled, mae_scaled, maxae_scaled = [], [], [], []
    scaler, X, y, preds_scaled, preds = nn.predictions(experiments[i], model,
                                                       difference_chosen = DIFFERENCE,
                                                      lag_chosen = LAG,
                                                      batch_size = BATCH_SIZE)
    all_scaler.append(scaler)
    all_X_values.append(X)
    all_y_values.append(y)
    all_predictions.append(preds)
    all_scaled_predictions.append(preds_scaled)
    
    gf.create_prediction_plot(experiments[i]['el_power'], preds, image_folder,
                              title = 'Using Difference as Output ',
                             specs = 'on ex_{0} with model trained on {1}'.format(names[i], train))

    gf.create_prediction_plot(y, preds_scaled[:,0], image_folder,
                              title = 'Difference in Power using Difference as Output ',
                             specs = 'differences on ex_{0} with model trained on {1}'.format(names[i], train))
    
    results_ex = gf.measure_difference(experiments[i]['el_power'], preds, should_print = False)
    results_scaled = gf.measure_difference(y, preds_scaled, should_print = False)
    rms.append(results_ex['RMSE'][0])
    r2.append(results_ex['R2'][0])
    mae.append(results_ex['MAE'][0])
    maxae.append(results_ex['MaxAE'][0])
    rms_scaled.append(results_scaled['RMSE'][0])
    r2_scaled.append(results_scaled['R2'][0])
    mae_scaled.append(results_scaled['MAE'][0])
    maxae_scaled.append(results_scaled['MaxAE'][0])
                   
    results.loc[names[i], 'rmse'] = rms
    results.loc[names[i], 'r2'] = r2
    results.loc[names[i], 'mae'] = mae
    results.loc[names[i], 'maxae'] = maxae
    results.loc[names[i], 'rmse_scaled'] = rms_scaled
    results.loc[names[i], 'r2_scaled'] = r2_scaled
    results.loc[names[i], 'mae_scaled'] = mae_scaled
    results.loc[names[i], 'maxae_scaled'] = maxae_scaled
    
    results.to_csv(image_folder + "99_results.csv", sep = "|", encoding = 'utf-8')

In [ ]:
gf.save_losses(history, image_folder)